In [10]:
from azureml.core import Dataset, Workspace, Model, Environment, Experiment
from azureml.core.resource_configuration import ResourceConfiguration
from azureml.core.conda_dependencies import CondaDependencies

from azureml.core import Datastore, Dataset

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.model import InferenceConfig

from azureml.data import DataType
from azureml.data.datapath import DataPath

from azureml.train.estimator import Estimator

from azureml.core import Webservice
from azureml.core.webservice import AciWebservice
from azureml.exceptions import WebserviceException

from azureml.core import Keyvault

import json

### Connect to the Workspace

In [3]:
ws = Workspace.from_config()
exp = Experiment(ws, "MaxFreezerTemperatureExceeded", _create_in_cloud=True)

In [2]:
# PARAMS
TIMESERIESLENGTH = 10
THRESHOLD = 180.0
TRAIN_DATA_SPLIT = 0.8

NUMBER_ESTIMATORS = 10

TRAIN_FOLDER_NAME = "train"
TRAIN_FILE_NAME = "train.py"

MODELNAME = "script-classifier"
SERVICENAME = "script-deployment"

MODELFOLDER = "outputs"
MODELFILENAME = "model.pkl"

## Prepare
### Access data (Blob Storage)

In [4]:
datastore = Datastore.get(ws, "sensordata")

datapath = DataPath(datastore=datastore, path_on_datastore="/processed/json/**")

dataset = Dataset.Tabular.from_json_lines_files(path=datapath, validate=True, include_path=False, set_column_types={"allevents": DataType.to_string(), "ConnectionDeviceID": DataType.to_string()}, partition_format='/{PartitionDate:yyyy/MM/dd}/')
dataset.register(workspace=ws, name="processed_json", description="Output from Stream Analytics", create_new_version=True)

print("dataset registered")

dataset registered


## Train & Test Model
### Create experiment
Define run parameters, compute target, environment and estimator


In [6]:
compute_target = ComputeTarget(ws, "freezertrain")

script_params = {
    "--timeserieslength": TIMESERIESLENGTH, 
    "--n_estimators": NUMBER_ESTIMATORS, 
    "--threshold": THRESHOLD, 
    "--model_folder": MODELFOLDER, 
    "--model_filename": MODELFILENAME, 
    '--train_data_split': TRAIN_DATA_SPLIT
}

est = Estimator(source_directory=TRAIN_FOLDER_NAME, 
                entry_script=TRAIN_FILE_NAME,
                script_params=script_params,
                inputs=[Dataset.get_by_name(ws, name="processed_json").as_named_input("rawdata")],
                compute_target=compute_target,
                conda_packages=["scikit-learn", "pandas", "cython"],
                pip_packages=['azureml-dataprep[pandas,fuse]==1.4.0', "azureml-defaults==1.1.5", "sktime"]
                )

### Run the experiment

In [7]:
run = exp.submit(est)
run.wait_for_completion(show_output=False, wait_post_processing=True)

## Register model

In [8]:
model = run.register_model(
    model_name=MODELNAME,  # Name of the registered model in your workspace.
    model_path=os.path.join(MODELFOLDER, MODELFILENAME),  # Local file to upload and register as a model.
    tags={
        "area": "freezerchain",
        "type": "classification",
        "purpose": "demonstration",
    },
    properties=None, 
    description="Sktime classifier to predict if freezer chain was interrupted.",
    datasets=[("training_data", dataset)], 
    resource_configuration=ResourceConfiguration(cpu=1, memory_in_gb=0.5))

## Build Image

In [20]:
environment = Environment("sktime_freezer_environment")

cd = CondaDependencies.create(
    conda_packages=["numpy", "cython"],
    pip_packages=[
        "azureml-defaults",
        "inference-schema[numpy-support]",
        "joblib==0.13.*",
        "scikit-learn==0.22.*",
        "pandas==0.23.*",
        "sktime",
    ],
)

environment.python.conda_dependencies = cd

environment.register(workspace=ws)

{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/base:intelmpi2018.3-ubuntu16.04",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "username": null
        },
        "enabled": false,
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "sktime_freezer_environment",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "anaconda",
                "conda-forge"
            ],
            "dependencies": [
                "python=3.6.2",
                {
                    "pip": [
  

## Deploy Webservice

In [77]:
try:
    Webservice(ws, SERVICENAME).delete()
    print("deleted existing Webservice.")
except WebserviceException:
    pass

inference_config = InferenceConfig(
    entry_script="score.py", 
    source_directory="deployment", 
    environment=environment)

aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1,)

service = Model.deploy(
    workspace=ws,
    name=SERVICENAME,
    models=[model],
    inference_config=inference_config,
    deployment_config=aci_config,
)

service.wait_for_deployment(show_output=True)
    
print(f"Scoring Uri: '{service.scoring_uri}'")

deleted existing Webservice.
Running..................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Scoring Uri: 'http://eeed9067-ff18-4cde-872d-4b2b7f0eb368.westeurope.azurecontainer.io/score'


## Bonus: Store Endpoint in Key Vault

In [78]:
keyvault = ws.get_default_keyvault()
keyvault.set_secret(name="webservice-name", value = SERVICENAME)
keyvault.set_secret(name="MLENDPOINT", value = service.scoring_uri)

# Test webservice deployment

In [24]:
keyvault = ws.get_default_keyvault()
service = Webservice(ws, keyvault.get_secret("webservice-name"))

In [26]:
with open("deployment/sample_data.json", "r") as fh:
    sample_data = json.loads(fh.read())

input_payload = json.dumps(sample_data)
output = service.run(input_payload)

# print(input_payload)
print(output)

{'result': True, 'ConnectionDeviceId': 'milkbottleEdge', 'timeCreatedStart': '2020-04-07T05:51:01.7604291Z', 'timeCreatedEnd': '2020-04-07T05:51:56.9882857Z', 'hasError': False, 'errorMessage': None}


In [27]:
print(service.get_logs())

2020-06-17T07:59:08,229133534+00:00 - iot-server/run 
2020-06-17T07:59:08,229183834+00:00 - rsyslog/run 
2020-06-17T07:59:08,230403340+00:00 - gunicorn/run 
2020-06-17T07:59:08,242731203+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_2852d664a66d5fa66708d97733323a4e/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_2852d664a66d5fa66708d97733323a4e/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_2852d664a66d5fa66708d97733323a4e/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_2852d664a66d5fa66708d97733323a4e/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_2852d664a66d5fa66708d97733323a4e/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
EdgeHubC